In [1]:
# libraries used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.neighbors import KNeighborsClassifier


from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
import matplotlib.patches as patches
from aif360.algorithms.preprocessing import Reweighing
#from packages import *
#from ml_fairness import *
import matplotlib.pyplot as plt
import seaborn as sns



from IPython.display import Markdown, display

import ipynbname
nb_fname = ipynbname.name()
nb_path = ipynbname.path()

import pickle

In [2]:
# This DataFrame is created to stock differents models and fair metrics that we produce in this notebook
algo_metrics = pd.DataFrame(columns=['model', 'fair_metrics', 'prediction', 'probs'])

def add_to_df_algo_metrics(algo_metrics, model, fair_metrics, preds, probs, name):
    return algo_metrics.append(pd.DataFrame(data=[[model, fair_metrics, preds, probs]], columns=['model', 'fair_metrics', 'prediction', 'probs'], index=[name]))

In [3]:
def fair_metrics(dataset, pred, pred_is_dataset=False):
    if pred_is_dataset:
        dataset_pred = pred
    else:
        dataset_pred = dataset.copy()
        dataset_pred.labels = pred
    
    cols = ['statistical_parity_difference', 'equal_opportunity_difference', 'average_abs_odds_difference',  'disparate_impact', 'theil_index']
    obj_fairness = [[0,0,0,1,0]]
    
    fair_metrics = pd.DataFrame(data=obj_fairness, index=['objective'], columns=cols)
    
    for attr in dataset_pred.protected_attribute_names:
        idx = dataset_pred.protected_attribute_names.index(attr)
        privileged_groups =  [{attr:dataset_pred.privileged_protected_attributes[idx][0]}] 
        unprivileged_groups = [{attr:dataset_pred.unprivileged_protected_attributes[idx][0]}] 
        
        classified_metric = ClassificationMetric(dataset, 
                                                     dataset_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)

        metric_pred = BinaryLabelDatasetMetric(dataset_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)

        acc = classified_metric.accuracy()

        row = pd.DataFrame([[metric_pred.mean_difference(),
                                classified_metric.equal_opportunity_difference(),
                                classified_metric.average_abs_odds_difference(),
                                metric_pred.disparate_impact(),
                                classified_metric.theil_index()]],
                           columns  = cols,
                           index = [attr]
                          )
        fair_metrics = fair_metrics.append(row)    
    
    fair_metrics = fair_metrics.replace([-np.inf, np.inf], 2)
        
    return fair_metrics

def plot_fair_metrics(fair_metrics):
    fig, ax = plt.subplots(figsize=(20,4), ncols=5, nrows=1)

    plt.subplots_adjust(
        left    =  0.125, 
        bottom  =  0.1, 
        right   =  0.9, 
        top     =  0.9, 
        wspace  =  .5, 
        hspace  =  1.1
    )

    y_title_margin = 1.2

    plt.suptitle("Fairness metrics", y = 1.09, fontsize=20)
    sns.set(style="dark")

    cols = fair_metrics.columns.values
    obj = fair_metrics.loc['objective']
    size_rect = [0.2,0.2,0.2,0.4,0.25]
    rect = [-0.1,-0.1,-0.1,0.8,0]
    bottom = [-1,-1,-1,0,0]
    top = [1,1,1,2,1]
    bound = [[-0.1,0.1],[-0.1,0.1],[-0.1,0.1],[0.8,1.2],[0,0.25]]

    display(Markdown("### Check bias metrics :"))
    display(Markdown("A model can be considered bias if just one of these five metrics show that this model is biased."))
    for attr in fair_metrics.index[1:len(fair_metrics)].values:
        display(Markdown("#### For the %s attribute :"%attr))
        check = [bound[i][0] < fair_metrics.loc[attr][i] < bound[i][1] for i in range(0,5)]
        display(Markdown("With default thresholds, bias against unprivileged group detected in **%d** out of 5 metrics"%(5 - sum(check))))

    for i in range(0,5):
        plt.subplot(1, 5, i+1)
        ax = sns.barplot(x=fair_metrics.index[1:len(fair_metrics)], y=fair_metrics.iloc[1:len(fair_metrics)][cols[i]])
        
        for j in range(0,len(fair_metrics)-1):
            a, val = ax.patches[j], fair_metrics.iloc[j+1][cols[i]]
            marg = -0.2 if val < 0 else 0.1
            ax.text(a.get_x()+a.get_width()/5, a.get_y()+a.get_height()+marg, round(val, 3), fontsize=15,color='black')

        plt.ylim(bottom[i], top[i])
        plt.setp(ax.patches, linewidth=0)
        ax.add_patch(patches.Rectangle((-5,rect[i]), 10, size_rect[i], alpha=0.3, facecolor="green", linewidth=1, linestyle='solid'))
        plt.axhline(obj[i], color='black', alpha=0.3)
        plt.title(cols[i])
        ax.set_ylabel('')    
        ax.set_xlabel('')

In [4]:
def get_fair_metrics_and_plot(data, model, plot=False, model_aif=False):
    pred = model.predict(data).labels if model_aif else model.predict(data.features)
    # fair_metrics function available in the metrics.py file
    fair = fair_metrics(data, pred)

    if plot:
        # plot_fair_metrics function available in the visualisations.py file
        # The visualisation of this function is inspired by the dashboard on the demo of IBM aif360 
        plot_fair_metrics(fair)
        display(fair)
    
    return fair

In [ ]:
data_orig_train = pd.read_pickle('../../Results/XGBoost/3-a-statistical-analysis-ml-workflow-of-titanic_Train.pkl')
data_orig_test = pd.read_pickle('../../Results/XGBoost/3-a-statistical-analysis-ml-workflow-of-titanic_Test.pkl')

In [146]:
import pandas as pd
import os
import glob

i = 0
# use glob to get all the csv files 
# in the folder
TM_dict = dict()
paths = ["..\\..\\Titanic\\Results\\XGBoost\\"]
for path in paths:
    csv_files = glob.glob(os.path.join(path, "*.pkl"))
    for f in csv_files:
        if i % 3 == 0 or i == 0:
            i = i + 1
            continue
        TM_dict[f] = pd.read_pickle(f)
        i = i + 1
        
i = 0
AC_dict = dict()    
paths = ["..\\..\\AdultNotebook\\Results\\XGBoost\\"]
for path in paths:
    csv_files = glob.glob(os.path.join(path, "*.pkl"))

    for f in csv_files:
        if i % 3 == 0 or i == 0:
            i = i + 1
            continue
        AC_dict[f] = pd.read_pickle(f)
        i = i + 1
        
i = 0
BM_dict = dict()
paths = ["..\\..\\BankMarketingNotebook\\Results\\XGBoost\\"]
for path in paths:
    csv_files = glob.glob(os.path.join(path, "*.pkl"))
    for f in csv_files:
        if i % 3 == 0 or i == 0:
            i = i + 1
            continue
            i = i + 1
        BM_dict[f] = pd.read_pickle(f)
        i = i + 1
        
i = 0
GC_dict = dict()
paths = ["..\\..\\GermanCredit\\Results\\XGBoost\\"]
for path in paths:
    print(path)
    csv_files = glob.glob(os.path.join(path, "*.pkl"))
    for f in csv_files:
        if i % 3 == 0 or i == 0:
            i = i + 1
            continue
        GC_dict[f] = pd.read_pickle(f)
        i = i + 1
    
    
models = [TM_dict, AC_dict, BM_dict, GC_dict]

..\..\GermanCredit\Results\XGBoost\


In [147]:
GC_dict.keys()

dict_keys(['..\\..\\GermanCredit\\Results\\XGBoost\\0-predicting-credit-risk-model-pipeline_Test.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\0-predicting-credit-risk-model-pipeline_Train.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\1-credit-risk-evaluation_Test.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\1-credit-risk-evaluation_Train.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\10-credit-risk-assessment_Test.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\10-credit-risk-assessment_Train.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\3-bank-credit-data-clustering-modelling_Test.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\3-bank-credit-data-clustering-modelling_Train.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\6-credit-risk-modelling-german-data_Test.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\6-credit-risk-modelling-german-data_Train.pkl', '..\\..\\GermanCredit\\Results\\XGBoost\\7-predicting-credit-risk-model-pipeline-60deb1_Test.pkl', '..\\..\\GermanCredit\\Results\

In [199]:
import ipynbname
nb_fname = ipynbname.name()
nb_path = ipynbname.path()

Overall_metrics = []

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from csv import writer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

for dataset in models:
    
    for i in range (0, len(dataset.keys()),2):
        print(i)
        data_orig_train = pd.read_pickle(list(dataset.keys())[i+1])
        data_orig_test = pd.read_pickle(list(dataset.keys())[i])
        
        X_train = data_orig_train.features
        y_train = data_orig_train.labels.ravel()

        X_test = data_orig_test.features
        y_test = data_orig_test.labels.ravel()
        
        
        final_metrics = []
        accuracy = []
        f1= []

        for i in range(0,10):

            model = AdaBoostClassifier(n_estimators= 200, base_estimator=DecisionTreeClassifier(max_depth = 6))

            mdl = model.fit(X_train, y_train)
            yy = mdl.predict(X_test)
            accuracy.append(accuracy_score(y_test, yy))
            f1.append(f1_score(y_test, yy))
            fair = get_fair_metrics_and_plot(data_orig_test, mdl)                           
            fair_list = fair.iloc[1].tolist()
            fair_list[3] = np.log(fair_list[3])
            final_metrics.append(fair_list)

        Overall_metrics.append(list(sum(x) for x in final_metrics))

0
2
4
6
8
10
0
2
4
6
8
10
0
2
4
6
8
10
0
2
4
6
8
10


In [197]:
import ipynbname
nb_fname = ipynbname.name()
nb_path = ipynbname.path()

Overall_metrics = []

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from csv import writer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

for dataset in models:
    
    for i in range (0, len(dataset.keys()),2):
        print(i)
        data_orig_train = pd.read_pickle(list(dataset.keys())[i+1])
        data_orig_test = pd.read_pickle(list(dataset.keys())[i])
        
        X_train = data_orig_train.features
        y_train = data_orig_train.labels.ravel()

        X_test = data_orig_test.features
        y_test = data_orig_test.labels.ravel()
        
        
        final_metrics = []
        accuracy = []
        f1= []


        model =  XGBClassifier(n_estimators= 100, max_depth = 12)

        mdl = model.fit(X_train, y_train)
        yy = mdl.predict(X_test)
        accuracy.append(accuracy_score(y_test, yy))
        f1.append(f1_score(y_test, yy))
        fair = get_fair_metrics_and_plot(data_orig_test, mdl)                           
        fair_list = fair.iloc[1].tolist()
        fair_list[3] = np.log(fair_list[3])
        final_metrics.append(fair_list)

        Overall_metrics.append(sum(final_metrics[0]))

0
2
4
6
8
10
0
2
4
6
8
10
0
2
4
6
8
10
0
2
4
6
8
10


In [198]:
Overall_metrics
    

[-2.6232558954364125,
 -2.768991219915285,
 -1.90241331974715,
 -2.5542659259507157,
 -1.8998885505291934,
 -2.0470509539848933,
 -1.16852915889595,
 -1.1196638544511306,
 -1.1657535293805215,
 -1.226294233515039,
 -1.1164914958279295,
 -1.1492301020397553,
 1.4193874325987281,
 0.12605755386485482,
 1.6340773242888629,
 0.74508017075025,
 1.4464837514843225,
 1.3609889451926762,
 0.2007528221188386,
 0.25006484704655285,
 0.033907917448325525,
 0.8508798814271157,
 -0.26064390563617634,
 0.015049577821735965]

In [194]:
import csv

with open("output_XGB.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(Overall_metrics)

In [120]:
from csv import writer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

final_metrics = []
accuracy = []
f1= []

for i in range(0,10):
    
    model = AdaBoostClassifier(n_estimators= 100, base_estimator=DecisionTreeClassifier(max_depth = 6))
    
    mdl = model.fit(X_train, y_train)
    yy = mdl.predict(X_test)
    accuracy.append(accuracy_score(y_test, yy))
    f1.append(f1_score(y_test, yy))
    fair = get_fair_metrics_and_plot(data_orig_test, mdl)                           
    fair_list = fair.iloc[1].tolist()
    fair_list[3] = np.log(fair_list[3])
    final_metrics.append(fair_list)


In [114]:
import statistics

total_bias = []
total_accuracy = []
total_f1 = []

for i in range(0, len(final_metrics)):
    total_bias.append(sum(final_metrics[i]))

In [104]:
print(statistics.mean(total_bias))
print(statistics.mean(accuracy))
print(statistics.mean(f1))

-0.011549209690540144
0.7353333333333334
0.8252532750235162


In [108]:
total_bias

[-0.06686382349233941,
 -0.032197113378255704,
 0.13329970781716197,
 -0.04039106355168129,
 -0.0903614754630932,
 -0.005973588955697912,
 -0.1342408232274386,
 -0.032078168116783656,
 -0.046484605931384565,
 0.027204245871282562]

In [109]:
from csv import writer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

final_metrics = []
accuracy = []
f1= []

for i in range(0,10):
    
    model = XGBClassifier(n_estimators= 100)
    
    mdl = model.fit(X_train, y_train)
    yy = mdl.predict(X_test)
    accuracy.append(accuracy_score(y_test, yy))
    f1.append(f1_score(y_test, yy))
    fair = get_fair_metrics_and_plot(data_orig_test, mdl)                           
    fair_list = fair.iloc[1].tolist()
    fair_list[3] = np.log(fair_list[3])
    final_metrics.append(fair_list)


In [118]:
from csv import writer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

final_metrics = []
accuracy = []
f1= []

for i in range(0,10):
    
    model = GradientBoostingClassifier(n_estimators=500, max_depth = 6)
    
    mdl = model.fit(X_train, y_train)
    yy = mdl.predict(X_test)
    accuracy.append(accuracy_score(y_test, yy))
    f1.append(f1_score(y_test, yy))
    fair = get_fair_metrics_and_plot(data_orig_test, mdl)                           
    fair_list = fair.iloc[1].tolist()
    fair_list[3] = np.log(fair_list[3])
    final_metrics.append(fair_list)


In [121]:
final_metrics

[[-0.07232441471571904,
  -0.01570897064902843,
  0.045733273203302116,
  -0.09745956268446926,
  0.18284246248733227],
 [-0.04703177257525082,
  0.013435303844563928,
  0.03323280343743351,
  -0.06349995754152825,
  0.1861513618079388],
 [-0.13858695652173914,
  -0.0062009094667218045,
  0.1508277274606336,
  -0.18701482722570426,
  0.15822187611741734],
 [-0.10827759197324416,
  -0.05363786688714345,
  0.08363711526175355,
  -0.13861317717128535,
  0.14662365468305202],
 [-0.0842391304347826,
  -0.05839189747829676,
  0.03677170631490595,
  -0.10945659287982967,
  0.15688678012369994],
 [-0.06751672240802675,
  -0.003513848697808908,
  0.05099934859132871,
  -0.0912675924365482,
  0.19135974138866416],
 [-0.08068561872909707,
  0.0059942124844977185,
  0.0787546819998246,
  -0.10658354367784008,
  0.16784233655869898],
 [-0.027591973244147083,
  -0.01705250103348488,
  0.0426171596076515,
  -0.03562168312256171,
  0.1690363168548286],
 [-0.05539297658862874,
  -0.004857379082265356,
